#Problem Statement
Objective: Implement a function quantum_sum to add two numbers using the Draper adder algorithm, incorporating the Quantum Fourier Transform (QFT) that is built from scratch, instead of using pre-built library functions.

##Code Overview
The code effectively implements the Draper adder algorithm using custom-built QFT functions. It consists of various functions that manage the initialization of qubits, execution of the QFT, evolution of quantum states, measurement, and the overall summation process. Below is a detailed breakdown of each part of the code.

In [2]:
!pip install qiskit
!pip install pylatexenc
!pip install qiskit_aer
!pip install qiskit_ibm_runtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=59ca7032b2e9302ac0397f26a700f9024cdcbd1315324095f7d040745f5b9a15
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from qiskit import QuantumCircuit,QuantumRegister,ClassicalRegister,transpile
from qiskit_aer import AerSimulator
import numpy as np
from qiskit.visualization import plot_histogram

##QFT Execution Function

Hadamard Gate: The function starts by applying a Hadamard gate to the n-th qubit, generating a superposition state. \\
Controlled Phase Rotations: Next, it applies controlled phase rotations to create the entangled state needed for QFT.

In [4]:
def executeQFT(qc, reg, n):
    # Executes the QTF of reg, one qubit a time
    # Apply one Hadamard gate to the n-th qubit of the quantum register reg, and
    # then apply repeated phase rotations with parameters being pi divided by
    # increasing powers of two

    qc.h(reg[n])
    for i in range(0, n):
        #cp(theta, control_qubit, target_qubit[, …])
        qc.cp(np.pi/float(2**(i+1)), reg[n-(i+1)], reg[n])

'''
qc: input quantum circuit
reg_a: first input register to execute QFT
reg_b: second input register to execute QFT
n: n-th qbit to apply hadamard and phase rotation
'''

'\nqc: input quantum circuit\nreg_a: first input register to execute QFT\nreg_b: second input register to execute QFT\nn: n-th qbit to apply hadamard and phase rotation\n'

##State Evolution Function

Purpose: This function evolves the state of reg_a to include the contributions from reg_b by applying additional controlled phase rotations. It effectively prepares the quantum state for addition in the Fourier domain

In [5]:
def evolveQFTStateSum(qc, reg_a, reg_b, n):
    # Evolves the state |F(psi(reg_a))> to |F(psi(reg_a+reg_b))> using the QFT
    # conditioned on the qubits of the reg_b.
    # Apply repeated phase rotations with parameters being pi divided by
    # increasing powers of two.

    l = len(reg_b)
    for i in range(n+1):
        if (n - i) > l - 1:
            pass
        else:
            #cp(theta, control_qubit, target_qubit[, ...])
            qc.cp(np.pi/float(2**(i)), reg_b[n-i], reg_a[n])

'''
qc: input quantum circuit
reg_a: first input register to execute QFT
reg_b: second input register to execute QFT
n: n-th qbit to apply hadamard and phase rotation

'''

'\nqc: input quantum circuit\nreg_a: first input register to execute QFT\nreg_b: second input register to execute QFT\nn: n-th qbit to apply hadamard and phase rotation\n\n'

##Implementing Inverse Quantum Fourier Transform (IQFT)

Reverse Phase Rotations: The inverse QFT undoes the transformations applied by QFT and returns the qubits to the computational basis. It applies controlled phase rotations with negative angles followed by a Hadamard gate.

In [6]:
def inverseQFT(qc, reg, n):
    # Executes the inverse QFT on a register reg.
    # Apply repeated phase rotations with parameters being pi divided by
    # decreasing powers of two, and then apply a Hadamard gate to the nth qubit
    # of the register reg.

    for i in range(n):
        #cp(theta, control_qubit, target_qubit[, ...])
        qc.cp(-1*np.pi/float(2**(n-i)), reg[i], reg[n])
    qc.h(reg[n])

##Performing the Quantum Adder Operation

Measurement: Measures the state of the qubits after performing the addition.
Simulation: Transpiles and runs the circuit on the AerSimulator, collecting results.

In [7]:
def qcadder(first, second, qc,result, cl, n):
    # Measure qubits
    for i in range(n+1):
        qc.measure(result[i], cl[i])

    # Set chosen backend and execute job
    backend=AerSimulator()
    new_circuit = transpile(qc, backend)
    job = backend.run(new_circuit)

    # Get results of program
    job_stats = job.result().get_counts()
    for key, value in job_stats.items():
        res = key
        prob = value
    return res, prob

##Initializing Qubits for Input Numbers

Purpose: Initializes the quantum registers based on the binary representation of the input integers. It flips the corresponding qubits to the |1⟩ state where necessary by applying X gate.

In [8]:
def initQubits(str, qc, reg, n):
    # Flip the corresponding qubit in register if a bit in the string is a 1
    for i in range(n):
        if str[i] == "1":
            qc.x(reg[n-(i+1)])

##Summation Function

QFT Application: Applies QFT to the first register a. \\
State Evolution: Evolves the state of register a by adding contributions from register b. \\
Inverse QFT: Applies the inverse QFT to convert the state back to the computational basis

In [9]:
import math
from qiskit import *

pie = np.pi
def sum(a, b, qc):
    n = len(a)-1
    # Compute the Fourier transform of register a
    for i in range(n+1):
        executeQFT(qc, a, n-i)

    # Add the two numbers by evolving the Fourier transform F(ψ(reg_a))>
    # to |F(ψ(reg_a+reg_b))>
    for i in range(n+1):
        evolveQFTStateSum(qc, a, b, n-i)

    # Compute the inverse Fourier transform of register a
    for i in range(n+1):
        inverseQFT(qc, a, i)
    return qc

##Output Function

Purpose: Converts the binary result back to decimal and prints the result of the addition along with its probability.

In [10]:
def printresult(first,second,res, prob,input1,input2):
  sum = int(res, 2)
  print( f'\n{first} + {second} = {res} with a probability of {prob}%' )
  print( f'\n{input1} + {input2} = {sum}' )

User Input: Accepts two positive integers from the user and converts them to binary format. \\
Register Initialization: Initializes the quantum and classical registers for the addition process. \\
Addition Execution: Executes the addition using the custom QFT and prints the result.

In [11]:
input1 = int(input("Enter a first positive integer:\n" ))
input2 = int(input("Enter a second positive integer:\n" ))
first = '{0:{fill}2b}'.format(input1, fill='0')
second = '{0:{fill}2b}'.format(input2, fill='0')
l1 = len(first)
l2 = len(second)
if l2>l1:
    first,second = second, first
    l2, l1 = l1, l2
second = ("0")*(l1-l2) + second
n = l1


a = QuantumRegister(n+1, "a")
b = QuantumRegister(n+1, "b")
cl = ClassicalRegister(n+1, "cl")
qc = QuantumCircuit(a, b, cl, name="qc")
initQubits(first, qc, a, n)
initQubits(second, qc, b, n)
adder_qc = sum(a,b,qc)
adder_qc.draw("mpl")
res , prob = qcadder(first, second, adder_qc,a, cl, n)
printresult(first,second,res, prob,input1,input2)

Enter a first positive integer:
123
Enter a second positive integer:
234

11101010 + 01111011 = 101100101 with a probability of 1024%

123 + 234 = 357
